In [1]:
%%capture
!pip install openai

In [10]:
import os
import openai
from google.colab import userdata

# Get the token from the environment variable
chave = userdata.get('OPENAIKEY') # Substitua pela sua chave de API

In [16]:
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=chave,
)

models = client.models.list()
for model in models.data:
    print(model.id)

gpt-4-turbo
gpt-4-turbo-2024-04-09
tts-1
tts-1-1106
chatgpt-4o-latest
dall-e-2
whisper-1
gpt-4-turbo-preview
gpt-4o-audio-preview
gpt-3.5-turbo-instruct
gpt-4o-audio-preview-2024-10-01
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
babbage-002
davinci-002
gpt-4o-realtime-preview-2024-10-01
dall-e-3
gpt-4o-realtime-preview
gpt-4o-mini
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-2024-08-06
gpt-4o
tts-1-hd
tts-1-hd-1106
gpt-4-1106-preview
text-embedding-ada-002
gpt-3.5-turbo-16k
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-1106
gpt-4-0613
gpt-4
gpt-3.5-turbo-instruct-0914


In [57]:
import requests
import re
import torch
from PIL import Image
from huggingface_hub import InferenceClient
from transformers import pipeline
import json
import pandas as pd
import base64
import requests
from PIL import Image
import io
from tqdm import tqdm
tqdm.pandas()

In [21]:
# Carregar o dataframe
df = pd.read_csv('https://raw.githubusercontent.com/jpcosta90/once-learning/refs/heads/main/data/one-page-document/one_page_document_complarison_sample.csv')
df = df[['file1','file2','same_pattern']]

In [58]:
# Função para codificar a imagem a partir de uma URL, com conversão de formato
def encode_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        # Abrir a imagem usando PIL
        image = Image.open(io.BytesIO(response.content))
        # Converter a imagem para JPEG em memória
        with io.BytesIO() as output_buffer:
            image.convert('RGB').save(output_buffer, format='JPEG')
            image_data = output_buffer.getvalue()
        # Codificar em base64
        return base64.b64encode(image_data).decode('utf-8')
    else:
        raise Exception(f"Não foi possível baixar a imagem de {image_url}, status code: {response.status_code}")

In [67]:
def classificar_documento(reference_image_url, image_url):

    reference_image_url = 'https://raw.githubusercontent.com/jpcosta90/once-learning/main/files/' + reference_image_url
    image_url = 'https://raw.githubusercontent.com/jpcosta90/once-learning/main/files/' + image_url

    # Get the base64 strings of the images
    image_prompt = encode_image(reference_image_url)
    base64_image = encode_image(image_url)

    # Enviar o prompt para a API
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": [
                {
                 "type": "text",
                 "text": '''
                          **Task:**

                          You are an AI assistant tasked with analyzing company budget documents to detect any that are out of standard.

                          **Instructions:**

                          - Analyze each provided document to determine if it conforms to the standard.
                          - The attached document is a reference of a **standard** document.
                          - Identify documents that are **out of standard** based on the following criteria:
                            - Lack of required information.
                            - Signs of unauthorized editing.
                            - Any other identifiable non-compliance issues.

                          **Output Requirements:**

                          - Return **only** a JSON object with the following attributes:
                            - `"status"`: either `"standard"` or `"out of standard"`.
                            - `"justification"`: a brief explanation indicating the type of non-compliance.

                          **Example Output:**

                          {"status": "out of standard", "justification": "The document is missing the total amount field."}

                          '''
                 },
                                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_prompt}"
                    }
                }
                ]
             },
            {"role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
                ]
             }
        ]
    )

    # Obter a resposta gerada
    classificacao = response.choices[0].message.content

    return classificacao

In [68]:
df['resultado'] = df.progress_apply(
    lambda row: classificar_documento(row['file1'], row['file2']), axis=1
)

100%|██████████| 20/20 [01:47<00:00,  5.38s/it]


In [69]:
# Salvar o dataframe atualizado
df.to_csv('gpt4_result_sample.csv', index=False)

In [ ]:
import os

# Get the token from the environment variable
gh_token = os.getenv("GH_TOKEN")

In [ ]:
import requests
import base64

# Leitura do arquivo CSV
with open('llama_result_sample.csv', 'rb') as f:
    content = f.read()

# Codificação do conteúdo em base64
content_b64 = base64.b64encode(content).decode('utf-8')

# Configurar os parâmetros da API
repo = 'jpcosta90/once-learning'
path = 'results/llama_result_sample.csv'
message = 'Adiciona sample_comparisons.csv'
token = gh_token

In [ ]:
# URL da API do GitHub
url = f'https://api.github.com/repos/{repo}/contents/{path}'

# Cabeçalhos da requisição
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

# Dados da requisição
data = {
    'message': message,
    'content': content_b64
}

# Enviar a requisição PUT
response = requests.put(url, headers=headers, json=data)

# Verificar a resposta
if response.status_code == 201:
    print('Arquivo enviado com sucesso!')
else:
    print(f'Erro ao enviar o arquivo: {response.content}')

Erro ao enviar o arquivo: b'{"message":"Bad credentials","documentation_url":"https://docs.github.com/rest","status":"401"}'


In [ ]:
# Verificar autenticação do token
response = requests.get('https://api.github.com/user', headers=headers)
if response.status_code == 200:
    print('Autenticação bem-sucedida!')
    print(f"Usuário autenticado: {response.json()['login']}")
else:
    print(f"Erro na autenticação: {response.content}")

Erro na autenticação: b'{"message":"Bad credentials","documentation_url":"https://docs.github.com/rest","status":"401"}'
